### Download the CELEBA dataset
- This notebook download CELEBA-HQ, and creates an HDF5 file with images and attributes at a specified size (e.g. 128x128x3).
- This is not the exact varient of CELEBA that I used for the trained model. But it should be similar enough to not make a difference. 
- Most of the code is based upon or modified from [https://github.com/tkarras/progressive_growing_of_gans](https://github.com/tkarras/progressive_growing_of_gans/blob/master/dataset_tool.py) that is part of the ProGAN github repo by Terro Karras at NVIDIA. 
    - This takes a long time. You can instead just grab the face images and skip all of the post-processing, or just grab a subset of the dataset. 

In [2]:
import os
import sys
import io
import glob
import pickle
import numpy as np
import scipy.ndimage
import PIL.Image
import h5py # conda install h5py
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.externals.joblib import Parallel, delayed

In [3]:
import hashlib
import bz2
import zipfile
import base64
import cryptography.hazmat.primitives.hashes
import cryptography.hazmat.backends
import cryptography.hazmat.primitives.kdf.pbkdf2
import cryptography.fernet

In [10]:
import dataset as ds


### Downlaod the dataset 

#### Create directory

In [50]:
dirpath = 'data/celebA/Img/img_celeba.7z/'
if os.path.exists(dirpath):
    print(dirpath + ' exists - skip')
else:
    os.makedirs(dirpath)

#### Download the Img folder

In [51]:
z7_files = [
    '0B7EVK8r0v71pQy1YUGtHeUM2dUE',
    '0B7EVK8r0v71peFphOHpxODd5SjQ',
    '0B7EVK8r0v71pMk5FeXRlOXcxVVU',
    '0B7EVK8r0v71peXc4WldxZGFUbk0',
    '0B7EVK8r0v71pMktaV1hjZUJhLWM',
    '0B7EVK8r0v71pbWFfbGRDOVZxOUU',
    '0B7EVK8r0v71pQlZrOENSOUhkQ3c',
    '0B7EVK8r0v71pLVltX2F6dzVwT0E',
    '0B7EVK8r0v71pVlg5SmtLa1ZiU0k',
    '0B7EVK8r0v71pa09rcFF4THRmSFU',
    '0B7EVK8r0v71pNU9BZVBEMF9KN28',
    '0B7EVK8r0v71pTVd3R2NpQ0FHaGM',
    '0B7EVK8r0v71paXBad2lfSzlzSlk',
    '0B7EVK8r0v71pcTFwT1VFZzkzZk0'
]

In [52]:
for dii, drive_id in enumerate(tqdm(z7_files)):
    attrs_filename = 'img_celeba.7z.'+str(dii+1).zfill(3)
    save_path = os.path.join(dirpath, attrs_filename)
    if os.path.exists(save_path):
        print('[*] {} already exists'.format(save_path))
    else:
        ds.download_file_from_google_drive(drive_id, save_path)

#### Unzip the image files

In [62]:
file_7z

'data/celebA/Img/img_celeba.7z/img_celeba.7z.006'

In [71]:
outpath = '/'.join(dirpath.split('/')[:-2])+'/'
outpath

'data/celebA/Img/'

In [ ]:
for file_7z in np.sort(glob.glob(dirpath+'img_celeba.7z*')):
    !7z x $file_7z  -o$outpath


7-Zip [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,24 CPUs)

Processing archive: data/celebA/Img/img_celeba.7z/img_celeba.7z.001

Extracting  img_celeba/000001.jpg
Extracting  img_celeba/000002.jpg
Extracting  img_celeba/000003.jpg
Extracting  img_celeba/000004.jpg
Extracting  img_celeba/000005.jpg
Extracting  img_celeba/000006.jpg
Extracting  img_celeba/000007.jpg
Extracting  img_celeba/000008.jpg
Extracting  img_celeba/000009.jpg
Extracting  img_celeba/000010.jpg
Extracting  img_celeba/000011.jpg
Extracting  img_celeba/000012.jpg
Extracting  img_celeba/000013.jpg
Extracting  img_celeba/000014.jpg
Extracting  img_celeba/000015.jpg
Extracting  img_celeba/000016.jpg
Extracting  img_celeba/000017.jpg
Extracting  img_celeba/000018.jpg
Extracting  img_celeba/000019.jpg
Extracting  img_celeba/000020.jpg
Extracting  img_celeba/000021.jpg
Extracting  img_celeba/000022.jpg
Extracting  img_celeba/000023.jpg
Extracting  

Extracting  img_celeba/000239.jpg
Extracting  img_celeba/000240.jpg
Extracting  img_celeba/000241.jpg
Extracting  img_celeba/000242.jpg
Extracting  img_celeba/000243.jpg
Extracting  img_celeba/000244.jpg
Extracting  img_celeba/000245.jpg
Extracting  img_celeba/000246.jpg
Extracting  img_celeba/000247.jpg
Extracting  img_celeba/000248.jpg
Extracting  img_celeba/000249.jpg
Extracting  img_celeba/000250.jpg
Extracting  img_celeba/000251.jpg
Extracting  img_celeba/000252.jpg
Extracting  img_celeba/000253.jpg
Extracting  img_celeba/000254.jpg
Extracting  img_celeba/000255.jpg
Extracting  img_celeba/000256.jpg
Extracting  img_celeba/000257.jpg
Extracting  img_celeba/000258.jpg
Extracting  img_celeba/000259.jpg
Extracting  img_celeba/000260.jpg
Extracting  img_celeba/000261.jpg
Extracting  img_celeba/000262.jpg
Extracting  img_celeba/000263.jpg
Extracting  img_celeba/000264.jpg
Extracting  img_celeba/000265.jpg
Extracting  img_celeba/000266.jpg
Extracting  img_celeba/000267.jpg
Extracting  im

Extracting  img_celeba/000483.jpg
Extracting  img_celeba/000484.jpg
Extracting  img_celeba/000485.jpg
Extracting  img_celeba/000486.jpg
Extracting  img_celeba/000487.jpg
Extracting  img_celeba/000488.jpg
Extracting  img_celeba/000489.jpg
Extracting  img_celeba/000490.jpg
Extracting  img_celeba/000491.jpg
Extracting  img_celeba/000492.jpg
Extracting  img_celeba/000493.jpg
Extracting  img_celeba/000494.jpg
Extracting  img_celeba/000495.jpg
Extracting  img_celeba/000496.jpg
Extracting  img_celeba/000497.jpg
Extracting  img_celeba/000498.jpg
Extracting  img_celeba/000499.jpg
Extracting  img_celeba/000500.jpg
Extracting  img_celeba/000501.jpg
Extracting  img_celeba/000502.jpg
Extracting  img_celeba/000503.jpg
Extracting  img_celeba/000504.jpg
Extracting  img_celeba/000505.jpg
Extracting  img_celeba/000506.jpg
Extracting  img_celeba/000507.jpg
Extracting  img_celeba/000508.jpg
Extracting  img_celeba/000509.jpg
Extracting  img_celeba/000510.jpg
Extracting  img_celeba/000511.jpg
Extracting  im

Extracting  img_celeba/000727.jpg
Extracting  img_celeba/000728.jpg
Extracting  img_celeba/000729.jpg
Extracting  img_celeba/000730.jpg
Extracting  img_celeba/000731.jpg
Extracting  img_celeba/000732.jpg
Extracting  img_celeba/000733.jpg
Extracting  img_celeba/000734.jpg
Extracting  img_celeba/000735.jpg
Extracting  img_celeba/000736.jpg
Extracting  img_celeba/000737.jpg
Extracting  img_celeba/000738.jpg
Extracting  img_celeba/000739.jpg
Extracting  img_celeba/000740.jpg
Extracting  img_celeba/000741.jpg
Extracting  img_celeba/000742.jpg
Extracting  img_celeba/000743.jpg
Extracting  img_celeba/000744.jpg
Extracting  img_celeba/000745.jpg
Extracting  img_celeba/000746.jpg
Extracting  img_celeba/000747.jpg
Extracting  img_celeba/000748.jpg
Extracting  img_celeba/000749.jpg
Extracting  img_celeba/000750.jpg
Extracting  img_celeba/000751.jpg
Extracting  img_celeba/000752.jpg
Extracting  img_celeba/000753.jpg
Extracting  img_celeba/000754.jpg
Extracting  img_celeba/000755.jpg
Extracting  im

Extracting  img_celeba/000968.jpg
Extracting  img_celeba/000969.jpg
Extracting  img_celeba/000970.jpg
Extracting  img_celeba/000971.jpg
Extracting  img_celeba/000972.jpg
Extracting  img_celeba/000973.jpg
Extracting  img_celeba/000974.jpg
Extracting  img_celeba/000975.jpg
Extracting  img_celeba/000976.jpg
Extracting  img_celeba/000977.jpg
Extracting  img_celeba/000978.jpg
Extracting  img_celeba/000979.jpg
Extracting  img_celeba/000980.jpg
Extracting  img_celeba/000981.jpg
Extracting  img_celeba/000982.jpg
Extracting  img_celeba/000983.jpg
Extracting  img_celeba/000984.jpg
Extracting  img_celeba/000985.jpg
Extracting  img_celeba/000986.jpg
Extracting  img_celeba/000987.jpg
Extracting  img_celeba/000988.jpg
Extracting  img_celeba/000989.jpg
Extracting  img_celeba/000990.jpg
Extracting  img_celeba/000991.jpg
Extracting  img_celeba/000992.jpg
Extracting  img_celeba/000993.jpg
Extracting  img_celeba/000994.jpg
Extracting  img_celeba/000995.jpg
Extracting  img_celeba/000996.jpg
Extracting  im

...

Extracting  img_celeba/052789.jpg
Extracting  img_celeba/052790.jpg
Extracting  img_celeba/052791.jpg
Extracting  img_celeba/052792.jpg
Extracting  img_celeba/052793.jpg
Extracting  img_celeba/052794.jpg
Extracting  img_celeba/052795.jpg
Extracting  img_celeba/052796.jpg
Extracting  img_celeba/052797.jpg
Extracting  img_celeba/052798.jpg
Extracting  img_celeba/052799.jpg
Extracting  img_celeba/052800.jpg
Extracting  img_celeba/052801.jpg
Extracting  img_celeba/052802.jpg
Extracting  img_celeba/052803.jpg
Extracting  img_celeba/052804.jpg
Extracting  img_celeba/052805.jpg
Extracting  img_celeba/052806.jpg
Extracting  img_celeba/052807.jpg
Extracting  img_celeba/052808.jpg
Extracting  img_celeba/052809.jpg
Extracting  img_celeba/052810.jpg
Extracting  img_celeba/052811.jpg
Extracting  img_celeba/052812.jpg
Extracting  img_celeba/052813.jpg
Extracting  img_celeba/052814.jpg
Extracting  img_celeba/052815.jpg
Extracting  img_celeba/052816.jpg
Extracting  img_celeba/052817.jpg
Extracting  im

Extracting  img_celeba/053030.jpg
Extracting  img_celeba/053031.jpg
Extracting  img_celeba/053032.jpg
Extracting  img_celeba/053033.jpg
Extracting  img_celeba/053034.jpg
Extracting  img_celeba/053035.jpg
Extracting  img_celeba/053036.jpg
Extracting  img_celeba/053037.jpg
Extracting  img_celeba/053038.jpg
Extracting  img_celeba/053039.jpg
Extracting  img_celeba/053040.jpg
Extracting  img_celeba/053041.jpg
Extracting  img_celeba/053042.jpg
Extracting  img_celeba/053043.jpg
Extracting  img_celeba/053044.jpg
Extracting  img_celeba/053045.jpg
Extracting  img_celeba/053046.jpg
Extracting  img_celeba/053047.jpg
Extracting  img_celeba/053048.jpg
Extracting  img_celeba/053049.jpg
Extracting  img_celeba/053050.jpg
Extracting  img_celeba/053051.jpg
Extracting  img_celeba/053052.jpg
Extracting  img_celeba/053053.jpg
Extracting  img_celeba/053054.jpg
Extracting  img_celeba/053055.jpg
Extracting  img_celeba/053056.jpg
Extracting  img_celeba/053057.jpg
Extracting  img_celeba/053058.jpg
Extracting  im

Extracting  img_celeba/053271.jpg
Extracting  img_celeba/053272.jpg
Extracting  img_celeba/053273.jpg
Extracting  img_celeba/053274.jpg
Extracting  img_celeba/053275.jpg
Extracting  img_celeba/053276.jpg
Extracting  img_celeba/053277.jpg
Extracting  img_celeba/053278.jpg
Extracting  img_celeba/053279.jpg
Extracting  img_celeba/053280.jpg
Extracting  img_celeba/053281.jpg
Extracting  img_celeba/053282.jpg
Extracting  img_celeba/053283.jpg
Extracting  img_celeba/053284.jpg
Extracting  img_celeba/053285.jpg
Extracting  img_celeba/053286.jpg
Extracting  img_celeba/053287.jpg
Extracting  img_celeba/053288.jpg
Extracting  img_celeba/053289.jpg
Extracting  img_celeba/053290.jpg
Extracting  img_celeba/053291.jpg
Extracting  img_celeba/053292.jpg
Extracting  img_celeba/053293.jpg
Extracting  img_celeba/053294.jpg
Extracting  img_celeba/053295.jpg
Extracting  img_celeba/053296.jpg
Extracting  img_celeba/053297.jpg
Extracting  img_celeba/053298.jpg
Extracting  img_celeba/053299.jpg
Extracting  im

Extracting  img_celeba/053512.jpg
Extracting  img_celeba/053513.jpg
Extracting  img_celeba/053514.jpg
Extracting  img_celeba/053515.jpg
Extracting  img_celeba/053516.jpg
Extracting  img_celeba/053517.jpg
Extracting  img_celeba/053518.jpg
Extracting  img_celeba/053519.jpg
Extracting  img_celeba/053520.jpg
Extracting  img_celeba/053521.jpg
Extracting  img_celeba/053522.jpg
Extracting  img_celeba/053523.jpg
Extracting  img_celeba/053524.jpg
Extracting  img_celeba/053525.jpg
Extracting  img_celeba/053526.jpg
Extracting  img_celeba/053527.jpg
Extracting  img_celeba/053528.jpg
Extracting  img_celeba/053529.jpg
Extracting  img_celeba/053530.jpg
Extracting  img_celeba/053531.jpg
Extracting  img_celeba/053532.jpg
Extracting  img_celeba/053533.jpg
Extracting  img_celeba/053534.jpg
Extracting  img_celeba/053535.jpg
Extracting  img_celeba/053536.jpg
Extracting  img_celeba/053537.jpg
Extracting  img_celeba/053538.jpg
Extracting  img_celeba/053539.jpg
Extracting  img_celeba/053540.jpg
Extracting  im

Extracting  img_celeba/053753.jpg
Extracting  img_celeba/053754.jpg
Extracting  img_celeba/053755.jpg
Extracting  img_celeba/053756.jpg
Extracting  img_celeba/053757.jpg
Extracting  img_celeba/053758.jpg
Extracting  img_celeba/053759.jpg
Extracting  img_celeba/053760.jpg
Extracting  img_celeba/053761.jpg
Extracting  img_celeba/053762.jpg
Extracting  img_celeba/053763.jpg
Extracting  img_celeba/053764.jpg
Extracting  img_celeba/053765.jpg
Extracting  img_celeba/053766.jpg
Extracting  img_celeba/053767.jpg
Extracting  img_celeba/053768.jpg
Extracting  img_celeba/053769.jpg


### Download the deltas files

In [ ]:
dirpath = 'data/celebA/Img/deltas/'

In [ ]:
delta_files = [['README.txt', '0B4qLcYyJmiz0U2hZTEJfaEZSMFE'],
                ['LICENSE.txt', '1y5k_G-3t-cjW01QMxXNfhcp95FCpts79'],
                ['img_list.txt', '0B4qLcYyJmiz0U25vdEVIU3NvNFk'],
                ['deltas00029.zip', '0B4qLcYyJmiz0U1pnMEI4WXN1S3M'],
                ['deltas00028.zip', '0B4qLcYyJmiz0U1BhWlFGRXc1aHc'],
                ['deltas00027.zip', '0B4qLcYyJmiz0U1BRYl9tSWFWVGM'],
                ['deltas00026.zip', '0B4qLcYyJmiz0U0tBanQ4cHNBUWc'],
                ['deltas00025.zip', '0B4qLcYyJmiz0U0lYX1J1Tk5vMjQ'],
                ['deltas00024.zip', '0B4qLcYyJmiz0U0YwQmluMmJuX2M'],
                ['deltas00023.zip', '0B4qLcYyJmiz0U05ZNG14X3ZjYW8'],
                ['deltas00022.zip', '0B4qLcYyJmiz0TzZySG9IWlZaeGc'],
                ['deltas00021.zip', '0B4qLcYyJmiz0TzBBNE8xbFhaSlU'],
                ['deltas00020.zip', '0B4qLcYyJmiz0TnJQSS1vZS1JYUE'],
                ['deltas00019.zip', '0B4qLcYyJmiz0TnBtdW83OXRfdG8'],
                ['deltas00018.zip', '0B4qLcYyJmiz0TmhIUGlVeE5pWjg'],
                ['deltas00017.zip', '0B4qLcYyJmiz0TmZqZXN3UWFkUm8'],
                ['deltas00016.zip', '0B4qLcYyJmiz0TmVkVGJmWEtVbFk'],
                ['deltas00015.zip', '0B4qLcYyJmiz0TmRZTmZyenViSjg'],
                ['deltas00014.zip', '0B4qLcYyJmiz0Tm5MSUp3ZTZ0aTg'],
                ['deltas00013.zip', '0B4qLcYyJmiz0TlpJU1pleF9zbnM'],
                ['deltas00012.zip', '0B4qLcYyJmiz0Tlhvdl9zYlV4UUE'],
                ['deltas00011.zip', '0B4qLcYyJmiz0TlNyLUtOTzk3QjQ'],
                ['deltas00010.zip', '0B4qLcYyJmiz0TlBCNFU3QkctNkk'],
                ['deltas00009.zip', '0B4qLcYyJmiz0Tl9wNEU2WWRqcE0'],
                ['deltas00008.zip', '0B4qLcYyJmiz0Tksyd21vRmVqamc'],
                ['deltas00007.zip', '0B4qLcYyJmiz0TjdaV2ZsQU94MnM'],
                ['deltas00006.zip', '0B4qLcYyJmiz0TjVkYkF4dTJRNUk'],
                ['deltas00005.zip', '0B4qLcYyJmiz0TjRzeWlhLVJIYk0'],
                ['deltas00004.zip', '0B4qLcYyJmiz0TjRxVkZ1NGxHTXc'],
                ['deltas00003.zip', '0B4qLcYyJmiz0TjRWUXVvM3hZZE0'],
                ['deltas00002.zip', '0B4qLcYyJmiz0TjNRV2dUamd0bEU'],
                ['deltas00001.zip', '0B4qLcYyJmiz0TjAwOTRBVmRKRzQ'],
                ['deltas00000.zip', '0B4qLcYyJmiz0TXdaTExNcW03ejA']
              ]

In [ ]:
for dii, (fname, drive_id) in enumerate(tqdm(z7_files)):
    save_path = os.path.join(dirpath, fname)
    if os.path.exists(save_path):
        print('[*] {} already exists'.format(save_path))
    else:
        ds.download_file_from_google_drive(drive_id, save_path)